# Imports and Config

In [1]:
import pandas as pd
import numpy as np
import re
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
from pathlib import Path
import voyageai
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
import pandas as pd

# Load file
df = pd.read_csv("../data/mcp/results/mcp_results_2026-02-18.csv")

# Count titles properly
title_counts = (
    df.groupby("title")
      .size()
      .reset_index(name="count")
)

# Ensure numeric
title_counts["count"] = pd.to_numeric(title_counts["count"], errors="coerce")

# Only duplicates
duplicates_only = (
    title_counts[title_counts["count"] > 1]
    .sort_values("count", ascending=False)
    .reset_index(drop=True)
)

duplicates_only["n_duplicates"] = duplicates_only["count"] - 1

print(f"Total unique titles: {df['title'].nunique()}")
print(f"Titles with duplicates: {len(duplicates_only)}")
print(f"Total duplicate rows: {duplicates_only['n_duplicates'].sum()}")

duplicates_only.head(25)

Total unique titles: 8358
Titles with duplicates: 951
Total duplicate rows: 1759


,title,count,n_duplicates
0,MCP Server,51,50
1,MCP,50,49
2,GitHub MCP Server,28,27
3,MCP Servers,23,22
4,mcp-server,21,20
5,Weather MCP Server,21,20
6,mcp,19,18
7,Linear MCP Server,18,17
8,mcp-servers,17,16
9,Jira MCP Server,15,14


# Config

In [13]:
# ============================================================
# Config — set this before running
# ============================================================
# Name of the raw scrape file to process (produced by mcp_scraper.py).
# Set to None to auto-detect the most recent mcp_scraped_*.csv in data/mcp/raw/.
RAW_SCRAPE_FILE = "mcp_scraped_2026-02-18.csv"  # e.g. "mcp_scraped_2026-02-17.csv"

# ---- Auto-detect if not specified ----
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "mcp" / "raw"
if RAW_SCRAPE_FILE is None:
    scrape_files = sorted(RAW_DIR.glob("mcp_scraped_*.csv"))
    if not scrape_files:
        raise FileNotFoundError("No mcp_scraped_*.csv files found in data/mcp/raw/")
    RAW_SCRAPE_FILE = scrape_files[-1].name
    print(f"Auto-detected scrape file: {RAW_SCRAPE_FILE}")

# Extract date from filename so all outputs share the same date stamp
date_str = RAW_SCRAPE_FILE.replace("mcp_scraped_", "").replace(".csv", "")
print(f"Processing: {RAW_SCRAPE_FILE}  (date_str = {date_str})")
print(f"Output MCP data file  : data/mcp/raw/mcp_data_{date_str}.csv")
print(f"Output MCP embeddings : data/embeddings/voyage_mcp_emb_{date_str}.npy")


Processing: mcp_scraped_2026-02-18.csv  (date_str = 2026-02-18)
Output MCP data file  : data/mcp/raw/mcp_data_2026-02-18.csv
Output MCP embeddings : data/embeddings/voyage_mcp_emb_2026-02-18.npy


# Data Formatting

## O*NET

In [14]:
# ---------- Load the two datasets ----------
tasks = pd.read_csv("../data/onet/tasks_to_dwas_v30.1.csv")  
dwa_ref = pd.read_csv("../data/onet/dwa_reference_v30.1.csv")       


# ---------- Merge on DWA ID ----------
merged = tasks.merge(
    dwa_ref[["DWA ID", "IWA ID", "IWA Title", "Element ID", "Element Name"]],
    on="DWA ID",
    how="left"
)

# ---------- Rename standardized columns ----------
merged = merged.rename(columns={
    "Task ID": "task_id",
    "Task": "task",
    "DWA ID": "dwa_id",
    "DWA Title": "dwa_title",
    "IWA ID": "iwa_id",
    "IWA Title": "iwa_title",
    "Element ID": "gwa_id",
    "Element Name": "gwa_title",
    "O*NET-SOC Code": "soc_code",
    "Title": "title"   
})

# ---------- Select relevant columns ----------
cols_to_keep = [
    "task_id", "task",
    "dwa_id", "dwa_title",
    "iwa_id", "iwa_title",
    "gwa_id", "gwa_title",
    "soc_code", "title"
]
merged = merged[[c for c in cols_to_keep if c in merged.columns]]

# Optional save full hierarchy for reference
# merged.to_csv("../data/onet_full_hierarchy.csv", index=False)

# Keep only the columns relevant for the cascading sheet
merged = merged[[
    "dwa_title", "task", "title"
]].drop_duplicates().reset_index(drop=True)

# Save 
merged.to_csv("../data/onet/onet_data.csv", index=False)



## MCP

In [15]:
# Load raw data
df = pd.read_csv(RAW_DIR / RAW_SCRAPE_FILE)

# --- 1. Remove emojis / non-ascii ---
def remove_non_ascii(text):
    if isinstance(text, str):
        return re.sub(r'[^\x00-\x7F]+', '', text)
    return text

for col in ["title", "description", "use_cases", "key_features"]:
    if col in df.columns:
        df[col] = df[col].apply(remove_non_ascii).str.strip()


# --- 2. Drop junk / empty text ---
def is_useless_desc(text):
    if not isinstance(text, str):
        return True
    txt = text.lower().strip()
    if len(txt.split()) < 5:
        return True
    bad_patterns = ["mirror", "demo", "test"]
    return any(p in txt for p in bad_patterns)

df["desc_is_useless"] = df["description"].apply(is_useless_desc)
df["has_use_cases"] = df["use_cases"].notna() & (df["use_cases"].str.strip() != "")

# keep if we have use cases OR meaningful description
df = df[(~df["desc_is_useless"]) | (df["has_use_cases"])].reset_index(drop=True)

# --- 3. Clean whitespace, collapse doubles ---
df = df.replace({r'\s+': ' '}, regex=True)

# --- 4. Standardize uploaded column (YYYY-MM-DD format) ---
def parse_relative_date(text):
    if not isinstance(text, str):
        return None

    m = re.search(r"(\d+)\s*(year|month|week|day)s?\s*ago", text.lower())
    if not m:
        return None

    num, unit = int(m.group(1)), m.group(2)
    now = datetime.now()

    if unit == "year":
        dt = now - timedelta(days=num * 365)
    elif unit == "month":
        dt = now - timedelta(days=num * 30)
    elif unit == "week":
        dt = now - timedelta(weeks=num)
    elif unit == "day":
        dt = now - timedelta(days=num)
    else:
        return None

    return dt.strftime("%Y-%m-%d")

df["uploaded_clean"] = df["uploaded"].apply(parse_relative_date)


def combine_text_with_features(desc, use, features):
    parts = []
    if isinstance(desc, str) and desc.strip():
        parts.append(f"Description: {desc.strip()}")
    if isinstance(features, str) and features.strip():
        parts.append(f"Key features: {features.strip()}")
    if isinstance(use, str) and use.strip():
        parts.append(f"Use cases: {use.strip()}")
    return "; ".join(parts)

df["text_for_llm"] = df.apply(
    lambda x: combine_text_with_features(
        x["description"], x["use_cases"], x["key_features"]
    ),
    axis=1
)

# --- 6. Drop rows with no usable text at all ---
df = df[df["text_for_llm"].str.strip() != ""]

# --- Deduplicate on final LLM input ---
before = len(df)
df = df.drop_duplicates(subset=["text_for_llm"]).reset_index(drop=True)
after = len(df)
print(f"Removed {before - after} duplicate rows")

# --- 7. Drop short junk entries ---
df["len_text"] = df["text_for_llm"].str.len()
df = df[df["len_text"] > 40]

# drop helper / raw columns
cols_to_drop = [
    "uploaded", "use_cases", "description", "key_features",
    "desc_is_useless", "has_use_cases", "len_text"
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# --- 8. Save cleaned dataset ---
out_file = f"mcp_data_{date_str}.csv"
df.to_csv(RAW_DIR / out_file, index=False)
print(f"Cleaned dataset saved: data/mcp/raw/{out_file}  ({len(df)} rows)")

Removed 0 duplicate rows
Cleaned dataset saved: data/mcp/raw/mcp_data_2026-02-18.csv  (1183 rows)


# Embedding

In [ ]:
# ============================================================
# Load cleaned MCP data + O*NET for embedding
# ============================================================

mcp_df = pd.read_csv(RAW_DIR / f"mcp_data_{date_str}.csv")
onet_df = pd.read_csv(DATA_DIR / "onet/onet_data.csv")

mcp_texts = mcp_df["text_for_llm"].tolist()
mcp_titles = mcp_df["title"].tolist()

# Unique DWA titles only (no point embedding duplicates)
dwa_titles_unique = onet_df["dwa_title"].dropna().drop_duplicates().reset_index(drop=True).tolist()

print(f"MCP texts to embed:     {len(mcp_texts):,}")
print(f"Unique DWA titles:      {len(dwa_titles_unique):,}")
print(f"O*NET rows (total):     {len(onet_df):,}")


## Voyage-4-large Embeddings

In [ ]:
# ============================================================
# Voyage-4-large embeddings
# Requires VOYAGE_API_KEY in your .env file
# ============================================================

VOYAGE_MODEL = "voyage-4-large"
VOYAGE_BATCH = 128
load_dotenv()
vo = voyageai.Client()  # reads VOYAGE_API_KEY from env

def voyage_embed_batched(texts, input_type="document", batch_size=VOYAGE_BATCH):
    """Embed texts in batches via Voyage API."""
    all_embs = []
    total_batches = (len(texts) - 1) // batch_size + 1
    for i in range(0, len(texts), batch_size):
        batch_num = i // batch_size + 1
        batch = texts[i : i + batch_size]
        result = vo.embed(batch, model=VOYAGE_MODEL, input_type=input_type)
        all_embs.extend(result.embeddings)
        print(f"  Batch {batch_num}/{total_batches} complete")
    return np.array(all_embs, dtype=np.float32)

emb_dir = DATA_DIR / "embeddings"
emb_dir.mkdir(exist_ok=True)

# ---------- MCP embeddings (generated per scrape batch) ----------
print(f"Embedding {len(mcp_texts):,} MCP texts with {VOYAGE_MODEL}...")
voyage_mcp_emb = voyage_embed_batched(mcp_texts, input_type="document")
print(f"  Shape: {voyage_mcp_emb.shape}")

mcp_emb_path = emb_dir / f"voyage_mcp_emb_{date_str}.npy"
np.save(mcp_emb_path, voyage_mcp_emb)
print(f"  Saved to: {mcp_emb_path.name}")

# ---------- DWA embeddings (generated once; reused for all future batches) ----------
dwa_emb_path = emb_dir / "voyage_dwa_emb.npy"
if dwa_emb_path.exists():
    print(f"\nDWA embeddings already exist — loading from disk.")
    voyage_dwa_emb = np.load(dwa_emb_path)
    print(f"  Shape: {voyage_dwa_emb.shape}")
else:
    print(f"\nGenerating DWA embeddings with {VOYAGE_MODEL}...")
    voyage_dwa_emb = voyage_embed_batched(dwa_titles_unique, input_type="document")
    print(f"  Shape: {voyage_dwa_emb.shape}")
    np.save(dwa_emb_path, voyage_dwa_emb)
    print(f"  Saved to: {dwa_emb_path.name}")

print(f"\nEmbeddings ready. Pass these paths to llm_classification.ipynb:")
print(f"  MCP_DATA_FILE = \"mcp_data_{date_str}.csv\"")
print(f"  MCP_EMB_FILE  = \"voyage_mcp_emb_{date_str}.npy\"")
